# Part I. ETL Pipeline for Pre-Processing the Files

## RUNNING THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Importing Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Getting current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creating a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob('/home/workspace/event_data/*.csv')
    #(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# checking the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. The Apache Cassandra coding portion of the project. 

## We now work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This will make a connection to a Cassandra instance our local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
#Creating a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Setting KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries.With Apache Cassandra we model the database tables on the queries we want to run.

# Query1:

**1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4**<br>
To answer this query, we need to select the name of the artist, song title and song's length from the music app history where sessionId = 338 and itemInSession = 4. To achieve this, we write the CQL query as below.

"SELECT artist, song_title, length FROM artist_table WHERE sessionId = 338 AND item_in_session = 4"

#### Steps involved:
* We create a table called artist_table.
* Our primary key will be sessionId and itemInSession, sessionId being the partition key and itemInSession being the clustering key. This is because we want to filter our table based on these values.
* The columns for the table will be sessionId,item_in_session, artist, song_title and length. We then insert values into them from the music app history.
* Finally we execute the query.

In [22]:
# Creating the table artists_table for the query1.
query = "CREATE TABLE IF NOT EXISTS artists_table"
query = query + "( sessionId int, item_in_session int, artist text, song_title text, length double,  \
                   PRIMARY KEY (sessionId, item_in_session))"
                   
try:
    session.execute(query)
except Exception as e:
    print(e)   
                    

In [23]:
# Code to set up the CSV file. The Apache Cassandra code below will insert values into the table created.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assigning the INSERT statements into the `query` variable
        query = "INSERT into artists_table (sessionId, item_in_session, artist, song_title, length)"
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        
        ## Assigning which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[3]), int(line[8]), line[0], line[9], float(line[5])))

In [26]:
## Query execution
query = "SELECT sessionId, artist, song_title, length FROM artists_table  "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.sessionId, row.artist, '____', row.song_title, '____', row.length)

AttributeError: 'Row' object has no attribute 'sessionId'

# Query 2:
**Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182** <br>
To answer this query, we need to select the name of the artist, song title and users' first and last names from the music app history where userid = 10 and sessionId = 182. To achieve this, we write the CQL query as below.

"SELECT artist, song_title, first_name, last_name FROM user_details WHERE userId = 10 AND sessionId = 182"

#### Steps involved:
* We create a table called user_details.
* Our composite partition key will be userID, sessionId and the clustering key will be itemInSession. We want to filter our table based on both userid and sessionId     because if we only use userId as partition key, the sessionid which belongs to the same user might be put into different nodes. With huge data volumes, this might     create an issue with performance.
* We also explicitly define itemInSession as CLUSTERING COLUMN to fulfill the sorting requirement.
* The columns for the table will be userId, sessionId, item_in_session, artist, song_title, first_name and last_name. We then insert values into them from the music     app history.
* Finally we execute the query.

In [34]:
## Creating the table user_details for query 2.
query = "CREATE TABLE IF NOT EXISTS user_details"
query = query + "( userId int, sessionId int, item_in_session int, artist text,\
                   song_title text, first_name text, last_name text,  \
                   PRIMARY KEY((userId, sessionId), item_in_session)) \
                   WITH CLUSTERING ORDER BY (item_in_session DESC)"
                   
try:
    session.execute(query)
except Exception as e:
    print(e) 

In [35]:
# Code to set up the CSV file. The Apache Cassandra code below will insert values into the table created.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assigning the INSERT statements into the `query` variable
        query = "INSERT into user_details( userId, sessionId,item_in_session, artist, song_title, first_name, last_name)"
        query = query + " VALUES(%s, %s, %s, %s, %s, %s, %s)"
        
        ## Assigning which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]),int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [37]:
# Query execution
query = "SELECT artist, song_title, first_name, last_name FROM user_details WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.song_title, '___', row.artist, '____', row.first_name, '____', row.last_name)

Catch You Baby (Steve Pitron & Max Sanna Radio Edit) ___ Lonnie Gordon ____ Sylvie ____ Cruz
Kilometer ___ Sebastien Tellier ____ Sylvie ____ Cruz
Greece 2000 ___ Three Drives ____ Sylvie ____ Cruz
Keep On Keepin' On ___ Down To The Bone ____ Sylvie ____ Cruz


# Query 3:
**Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'**<br>
To answer this query, we need to select the users' first and last names from the music app history who listened to a song named 'All Hands Against His Own'. To achieve this, we write the CQL query as below.

"SELECT first_name, last_name FROM music_history WHERE song_title = 'All Hands Against His Own'"

#### Steps involved:
* We create a table called music_history.
* Our composite partition key will be song_title and userId. We want to filter our table based on both song_title and userId because if we only use song_title as partition key, results of users with same names will be omitted.
* The columns for the table will be song_title, userId, first_name and last_name. We then insert values into them from the music app history.
* Finally we execute the query.

In [99]:
## Creating the table music_history for query 3.
query = "CREATE TABLE IF NOT EXISTS music_history"
query = query + "(song_title text, userId int, first_name text, last_name text,\
                   PRIMARY KEY(song_title, userId))"
                   
try:
    session.execute(query)
except Exception as e:
    print(e)

In [100]:
# Code to set up the CSV file. The Apache Cassandra code below will insert values into the table created.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assigning the INSERT statements into the `query` variable
        query = "INSERT into music_history( song_title, userId, first_name, last_name)"
        query = query + " VALUES(%s, %s, %s, %s)"
        
        ## Assigning which column element should be assigned for each column in the INSERT statement.
    session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [101]:
## Query execution
query = "SELECT first_name, last_name FROM music_history WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, '____', row.last_name)

### Droping the tables before closing out the sessions

In [48]:
query = "DROP TABLE artists_table"
query = "DROP TABLE user_details"
query = "DROP TABLE song_history"

### Closing the session and cluster connection¶

In [50]:
session.shutdown()
cluster.shutdown()